### Introduction to Jupyter Notebooks

[Jupyter Notebooks](https://jupyter.org/) are a popular tool for interactive computing used by data professionals. Notebooks provide a versatile environment for writing and executing code, creating visualizations, and documenting your work.

#### Running Individual Cells

Jupyter Notebooks are organized into cells. Each cell can contain code or text. To run an individual code cell:

1. Click on the cell you want to run. The cell will be highlighted.
2. Press `Shift + Enter` on your keyboard, or click the "Run" button in the toolbar at the top. The code in the cell will execute, and any output (such as printed text or graphs) will be displayed immediately below the cell.
3. You can run cells in any order, making it easy to experiment and iterate with your code. Note that the notebook here expects cells to be executed in order to produce the intended results.

#### Running All Cells in Sequence

To run all cells in the notebook in sequence:

1. Go to the "Cell" menu at the top.
2. Select "Run All." This will execute all the cells in the notebook from top to bottom.
3. Running all cells is useful when you want to ensure that your notebook runs correctly from start to finish.

#### Downloading a Jupyter Notebook

When you want to save a local copy of the notebook:

1. Go to the "File" menu.
2. Select "Download" - this will save your notebook in the native Jupyter Notebook format (**.ipynb**). and the notebook will be downloaded to your computer.


In [ ]:
base_url = "http://127.0.0.1:5000/"

In [ ]:
latitude = 65.1
longitude = -146.6

In [ ]:
import micropip
await micropip.install('pyodide-http')
import pyodide_http
pyodide_http.patch_all()

In [ ]:
import pandas as pd
precip_df = pd.read_csv(f"{base_url}precipitation/frequency/point/{latitude}/{longitude}?format=csv", header=9)
precip_df

In [ ]:
duration_hrs = {
    "60m": 1,
    "2h": 2,
    "3h": 3,
    "6h": 6,
    "12h": 12,
    "24h": 24,
    "2d": 48,
    "3d": 72,
    "4d": 96,
    "7d": 168,
    "10d": 240,
    "20d": 480,
    "30d": 720,
    "45d": 1080,
    "60d": 1440,
}

In [ ]:
def mm_to_inches(df, columns_to_convert):
    """
    Convert precipitation amounts from millimeters (mm) to inches and replace the original columns in a DataFrame.

    Args:
        df (pd.DataFrame): DataFrame containing the precipitation columns.
        columns_to_convert (list): column names to convert from mm to inches.

    Returns:
        pd.DataFrame: DataFrame with converted columns.
    """
    for col in columns_to_convert:
        df[col] = round(df[col] * 0.0393701, 3)  # 1 mm = 0.0393701 inches
    
    return df

In [ ]:
def inches_to_inches_per_hour_intensity(df, columns_to_convert, tc):
    """
    Normalize precipitation amounts to inches per hour based on the duration.
    Args:
        df (pd.DataFrame): DataFrame containing the precipitation columns.
        columns_to_convert (list): columns to convert from inches to inches/hour.
        tc (str): Time of concentration, effectively this is the "duration"

    Returns:
        pd.DataFrame: DataFrame with converted columns.
    """
    for col in columns_to_convert:
        df[col] = round(df[col] / duration_hrs[tc], 3)
    
    return df

In [ ]:
await micropip.install("ipywidgets")
import ipywidgets as widgets
from IPython.display import display

# define the input widgets
return_interval_dropdown = widgets.Dropdown(
    options=[2, 5, 10, 25, 50, 100, 200, 500, 1000],
    value=100,
    description="Return Interval (years):",
    style={"description_width": "initial"}
)

# dropdown widget for selecting time of concentration (tc)
duration_hrs = {
    "60m": 1,
    "2h": 2,
    "3h": 3,
    "6h": 6,
    "12h": 12,
    "24h": 24,
    "2d": 48,
    "3d": 72,
    "4d": 96,
    "7d": 168,
    "10d": 240,
    "20d": 480,
    "30d": 720,
    "45d": 1080,
    "60d": 1440,
}
tc_dropdown = widgets.Dropdown(
    options=list(duration_hrs.keys()),
    value="24h",
    description="Time of Concentration (duration)",
    style={"description_width": "initial"}
)

drainage_area_acres_slider = widgets.FloatSlider(
    value=100.0, min=1.0, max=1000.0, step=1.0, description="Drainage Area (acres):",
    style={"description_width": "initial"}
)
c_runoff_coeff_slider = widgets.FloatSlider(
    value=0.3, min=0.01, max=1.0, step=0.01, description="Runoff Coefficient (C):",
    style={"description_width": "initial"}
)

# output widget for displaying the result
output = widgets.Output()
output.layout.height = "350px"

df = precip_df.copy()

# define the function to compute and display the result
def update_result(change):
    with output:
        output.clear_output()
        
        selected_tc = tc_dropdown.value
        precip = df[(df.return_interval == return_interval_dropdown.value) & (df.duration == selected_tc)].copy()
        precip = mm_to_inches(precip, ["pf", "pf_lower", "pf_upper"])
        precip = inches_to_inches_per_hour_intensity(precip, ["pf", "pf_lower", "pf_upper"], selected_tc)
        precip[f"Design Q{return_interval_dropdown.value} (cfs)"] = (c_runoff_coeff_slider.value * precip["pf"] * drainage_area_acres_slider.value).round(1)
        result_df = precip.set_index("era").sort_index()[["model", f"Design Q{return_interval_dropdown.value} (cfs)"]]
        
        # Display the resulting DataFrame
        display(result_df)

# attach change handler to the widgets
return_interval_dropdown.observe(update_result, names="value")
tc_dropdown.observe(update_result, names="value")
drainage_area_acres_slider.observe(update_result, names="value")
c_runoff_coeff_slider.observe(update_result, names="value")

# display the input widgets and output
display(return_interval_dropdown, tc_dropdown, drainage_area_acres_slider, c_runoff_coeff_slider)
display(output)

# display the initial result
update_result(None)
